# Оценка рыночной стоимости мотоцикла с пробегом

## Выполнили: Болгарин Максим, Моисеенков Павел. BD-11

<br>

**Цель проекта:** проанализировать факторы, влияющие на рыночную стоимость подержанных мотоциклов, и научиться её предсказывать. Какую пользу можно вынести из этого простому человеку? Зная, как образуется стоимость мотоцикла, владелец подобного транспортного средства может подобрать подходящую для него цену для продажи. С другой стороны, будущий владелец сможет оценить адекватность заинтересовавшего его предложения.

**План выполнения:**
1. Поиск данных, формирование датасета
    * Скачивание и парсинг данных.
    * Формирование датасета.
2. Первичный анализ данных
    * Изучение данных: определение смысла признаков, типов данных, наличия пропусков и т.д.
3. Предобработка данных
    * Очистка и отбор данных, выполнение необходимых преобразований.
4. Визуальный анализ данных
    * Поиск скрытых особенностей в данных, анализ корреляций.
5. Применение моделей анализа данных для предсказания стоимости
    * Тестирование нескольких простых моделей, сравнение качества, выбор baseline.
    * Обучение и проверка модели, показывающей наилучшее качество.
6. Вывод

<br>

### 1. Поиск данных, формирование датасета

Источником данных послужил архив объявлений на сайте moto.drom.ru

Всего было обработано около 7000 объявлений из Москвы. Скачивание производилось в [другом Jupyter Notebook](https://github.com/maxbolgarin/datamining_project/blob/master/project/parce.ipynb). На выходе мы имеем .csv файл, с ним нам и предстоит работать.

#### Подключим необходимые библиотеки

In [3]:
import numpy as np
import pandas as pd

#### Теперь загрузим данные

In [4]:
motorcycles = pd.read_csv('data/motorcycles.csv', index_col='id')

In [5]:
motorcycles.sample(5)

,price,model,mileage,motorcycle_class,year,engine_capacity,engine_strokes,damaged,documents,city,date
id,,,,,,,,,,,
4311,419 000₽,Yamaha FZ 8 SA,9 304 км,None,2012,798 куб. см.,4х тактный,Исправен,Есть ПТС,Москва,1 октября 2015
3666,500 000₽,Triumph Street Triple 675 ABS,5 400 км,None,2014,675 куб. см.,4х тактный,Исправен,Есть ПТС,Москва,3 апреля 2016
763,149 000₽,Yamaha YBR 125,1 км,Классика,2016,125 куб. см.,4х тактный,None,Есть ПТС,Москва,24 февраля
3622,200 000₽,Honda CBR 900RR,41 000 км,None,1998,918 куб. см.,4х тактный,Исправен,Есть ПТС,Москва,10 апреля 2016
5736,460 000₽,Harley-Davidson,850 км,None,2011,1 200 куб. см.,4х тактный,Исправен,Есть ПТС,Москва,5 марта 2014


In [6]:
motorcycles.shape

(6848, 11)

Видим, что данные успешно загрузились и выглядят вполне адекватно. Можем работать дальше.

<br>

### 2. Первичный анализ данных

Посмотрим общую информацию по датасету:

In [7]:
motorcycles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6848 entries, 1 to 7100
Data columns (total 11 columns):
price               6848 non-null object
model               6848 non-null object
mileage             6848 non-null object
motorcycle_class    6848 non-null object
year                6848 non-null object
engine_capacity     6848 non-null object
engine_strokes      6848 non-null object
damaged             6848 non-null object
documents           6848 non-null object
city                6848 non-null object
date                6848 non-null object
dtypes: object(11)
memory usage: 642.0+ KB


In [8]:
motorcycles.nunique()

price               1059
model               2284
mileage             1958
motorcycle_class      15
year                  56
engine_capacity      348
engine_strokes         3
damaged                3
documents              3
city                   1
date                2133
dtype: int64

In [9]:
motorcycles.drop_duplicates().shape

(6797, 11)

Теперь посмотрим на некоторые столбцы, вызывающие вопросы:

**price:**

In [30]:
motorcycles.price.unique()

array(['300 000₽', '98 000₽', '595 000₽', ..., '90₽', '9 000₽', '100₽'],
      dtype=object)

**motorcycle_class:**

In [13]:
motorcycles.motorcycle_class.value_counts() / motorcycles.motorcycle_class.size

None               0.518400
Чоппер             0.178592
Классика           0.067465
Спортивный         0.053154
Круизер            0.040888
Спорт-турист       0.037967
Эндуро             0.025263
Питбайк            0.024971
Туристический      0.018692
Стритфайтер        0.011536
Кроссовый          0.008470
Кастом             0.007155
Мотард             0.003213
Детский            0.002482
Трайк (трицикл)    0.001752
Name: motorcycle_class, dtype: float64

**engine_capacity:**

In [14]:
motorcycles.engine_capacity.value_counts()[:9] / motorcycles.engine_capacity.size

400 куб. см.      0.110981
600 куб. см.      0.097985
250 куб. см.      0.088347
1 000 куб. см.    0.049504
650 куб. см.      0.045999
750 куб. см.      0.036069
1 200 куб. см.    0.035193
1 300 куб. см.    0.032856
125 куб. см.      0.027599
Name: engine_capacity, dtype: float64

**engine_strokes:**

In [10]:
motorcycles.engine_strokes.value_counts() / motorcycles.engine_strokes.size

4х тактный    0.863902
None          0.081484
2х тактный    0.054614
Name: engine_strokes, dtype: float64

**damaged:**

In [11]:
motorcycles.damaged.value_counts() / motorcycles.damaged.size

Исправен      0.968166
None          0.019568
Неисправен    0.012266
Name: damaged, dtype: float64

**documents:**

In [16]:
motorcycles.documents.value_counts() / motorcycles.documents.size

Есть ПТС    0.908732
Без ПТС     0.089223
None        0.002044
Name: documents, dtype: float64

Теперь пройдемся по столбцам и, на основе уже имеющейся информации, дадим им некоторое описание:
1. **id:**
2. **price:**